In [113]:
# Import necessary modules and functions

import pandas as pd
import seaborn as sns
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 16, 8

from scipy.stats import norm
from scipy import stats

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix

import warnings
#warnings.filterwarnings('ignore')
from datetime import datetime

In [114]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [115]:
# Import the 'HousePrices.csv' file into a Pandas DataFrame.

db = pd.read_csv("/content/drive/MyDrive/IDS/KS_train_data.csv",delimiter=',')

In [116]:
db.head()

,project_id,backers_count,blurb,category,converted_pledged_amount,country,created_at,currency,deadline,fx_rate,goal,launched_at,name,pledged,staff_pick,usd_pledged,location,funded,subcategory,project_url,reward_url
0,KS_000000,80,I will be an artist-in-residence at Elsewhere ...,art,3596,US,1325980505,USD,1334866560,1.0,2800.0,1332969260,Elsewhere Studios artist-in-residency program!,3596.0,False,3596.0,"Paonia, CO",True,mixed media,https://www.kickstarter.com/projects/hilaryeme...,https://www.kickstarter.com/projects/hilaryeme...
1,KS_000001,82,1000 Artists is a public art-making installati...,art,4586,US,1330926084,USD,1336440145,1.0,4500.0,1332984145,1000 Artists: Presidential Inauguration 2013,4586.0,False,4586.0,"Washington, DC",True,public art,https://www.kickstarter.com/projects/17146650/...,https://www.kickstarter.com/projects/17146650/...
2,KS_000002,21,"The Sequel to ‘My Favorite Machine”, ""MyPhoneH...",art,5217,US,1332382894,USD,1338175739,1.0,5000.0,1332991739,MyPhoneHenge,5217.0,False,5217.0,"Frisco, TX",True,sculpture,https://www.kickstarter.com/projects/belveal/m...,https://www.kickstarter.com/projects/belveal/m...
3,KS_000003,37,A film exploring the role and value of art edu...,art,7160,GB,1332342451,USD,1334806096,1.0,6500.0,1332991696,Walk-Through,7160.0,False,7160.0,"Glasgow, UK",True,art,https://www.kickstarter.com/projects/107813091...,https://www.kickstarter.com/projects/107813091...
4,KS_000004,153,We need to build a kitchen for Habitable Space...,art,15445,US,1328562617,USD,1335584240,1.0,15000.0,1332992240,A kitchen for Habitable Spaces,15445.0,False,15445.0,"Kingsbury, TX",True,public art,https://www.kickstarter.com/projects/104409276...,https://www.kickstarter.com/projects/104409276...


In [117]:
dbFunded = db.loc[db['funded'] == True]

In [118]:
# get top 10 most frequent funded coutries
n = 10
topc=dbFunded['country'].value_counts()[:n].index.tolist()
topc

['US', 'GB', 'CA', 'AU', '?', 'DE', 'FR', 'SE', 'MX', 'NL']

In [119]:
dbc2 = db.copy()

In [120]:
dbc2.columns

Index(['project_id', 'backers_count', 'blurb', 'category',
       'converted_pledged_amount', 'country', 'created_at', 'currency',
       'deadline', 'fx_rate', 'goal', 'launched_at', 'name', 'pledged',
       'staff_pick', 'usd_pledged', 'location', 'funded', 'subcategory',
       'project_url', 'reward_url'],
      dtype='object')

In [121]:
dbc2.drop(['backers_count', 'blurb',
       'converted_pledged_amount', 'created_at',
       'deadline', 'fx_rate', 'launched_at', 'name', 'pledged',
     'usd_pledged','project_url', 'reward_url','location', 'subcategory'], axis = 1, inplace=True)

In [122]:
dbc2.columns

Index(['project_id', 'category', 'country', 'currency', 'goal', 'staff_pick',
       'funded'],
      dtype='object')

In [123]:
# if country not part of top10 then rename to 'other'
dbc2.loc[~dbc2.country.isin(topc), 'country'] = 'other'

In [124]:
dbc2.country

0        US
1        US
2        US
3        GB
4        US
         ..
99995    US
99996    US
99997    US
99998    US
99999    GB
Name: country, Length: 100000, dtype: object

In [125]:
dbc2.country.unique()

array(['US', 'GB', 'CA', 'other', 'AU', 'FR', 'NL', 'MX', 'DE', '?', 'SE'],
      dtype=object)

In [126]:
dbc2.columns

Index(['project_id', 'category', 'country', 'currency', 'goal', 'staff_pick',
       'funded'],
      dtype='object')

In [127]:
list(set(dbc2.country) - set(topc))

['other']

In [128]:
# one hot encoding ['category', 'currency','staff_pick','country']
db_hot = pd.get_dummies(dbc2, drop_first=True, columns=['category', 'currency','staff_pick','country'])

In [129]:
X_train, X_test, y_train, y_test = train_test_split(db_hot.drop(['project_id','funded'], axis=1), db_hot['funded'])

In [130]:
LogReg = LogisticRegression(solver='lbfgs')
LogReg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [131]:
print(LogReg.score(X_train,y_train))

0.41485333333333335


The model with the derived feature coutry is not performing very well with accoracy score of 0.41564

In [132]:
# creating a new db to experiment with
dbc3 = db.copy()

In [133]:
dbc3.columns

Index(['project_id', 'backers_count', 'blurb', 'category',
       'converted_pledged_amount', 'country', 'created_at', 'currency',
       'deadline', 'fx_rate', 'goal', 'launched_at', 'name', 'pledged',
       'staff_pick', 'usd_pledged', 'location', 'funded', 'subcategory',
       'project_url', 'reward_url'],
      dtype='object')

In [134]:
# selecting the columns that will b used
dbc3 = dbc3[['project_id', 'category', 'currency', 'goal', 'staff_pick', 'funded',
       'subcategory']]

In [135]:
dbc3.columns

Index(['project_id', 'category', 'currency', 'goal', 'staff_pick', 'funded',
       'subcategory'],
      dtype='object')

In [136]:
# one hot encoding the features
db_hot = pd.get_dummies(dbc3, drop_first=True, columns=['category', 'currency', 'goal', 'staff_pick', 'subcategory'])

In [137]:
# performing the train test split
X_train, X_test, y_train, y_test = train_test_split(db_hot.drop(['project_id','funded'], axis=1), db_hot['funded'])

In [138]:
LogReg = LogisticRegression(solver='lbfgs')
LogReg.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [139]:
print(LogReg.score(X_train,y_train))

0.7940666666666667


a big improvement is observed. Probably the feature 'goal' has made an impact


In [140]:
dbc4 = db.copy()

In [141]:
dbc4 = dbc4[['project_id', 'category', 'currency', 'goal', 'staff_pick', 'funded',
       'subcategory', 'location']]

In [142]:
dbc4.columns

Index(['project_id', 'category', 'currency', 'goal', 'staff_pick', 'funded',
       'subcategory', 'location'],
      dtype='object')

In [143]:
#now we wil attempt to create new feattre drived from location
# get top 10 most frequent funded location
n = 10
topl=dbFunded['location'].value_counts()[:n].index.tolist()
topl

['New York, NY',
 'Los Angeles, CA',
 'Brooklyn, NY',
 'San Francisco, CA',
 'Chicago, IL',
 'London, UK',
 'Portland, OR',
 'Seattle, WA',
 'Austin, TX',
 'Boston, MA']

In [144]:
dbc4.loc[~dbc4.location.isin(topl), 'location'] = 'other'

In [145]:
dbc4.location.unique()

array(['other', 'Chicago, IL', 'Seattle, WA', 'New York, NY',
       'Boston, MA', 'Brooklyn, NY', 'Portland, OR', 'San Francisco, CA',
       'Austin, TX', 'Los Angeles, CA', 'London, UK'], dtype=object)

In [146]:
#train with new features

In [147]:
db_hot = pd.get_dummies(dbc4, drop_first=True, columns=['category', 'currency', 'goal', 'staff_pick','subcategory', 'location'])

In [148]:
X_train, X_test, y_train, y_test = train_test_split(db_hot.drop(['project_id','funded'], axis=1), db_hot['funded'])

In [149]:
LogReg = LogisticRegression(solver='lbfgs')
LogReg.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [150]:
print(LogReg.score(X_train,y_train))

0.7956933333333334


In [151]:
dbc5 = db.copy()

In [152]:
dbc5 = dbc5[['project_id', 'category', 'currency', 'goal', 'staff_pick', 'funded',
       'subcategory', 'location']]

In [153]:
dbc5.columns

Index(['project_id', 'category', 'currency', 'goal', 'staff_pick', 'funded',
       'subcategory', 'location'],
      dtype='object')

In [154]:
#now we wil attempt to create new feattre drived from location
# get top 10 most frequent funded location
n = 20
topl=dbFunded['location'].value_counts()[:n].index.tolist()
topl

['New York, NY',
 'Los Angeles, CA',
 'Brooklyn, NY',
 'San Francisco, CA',
 'Chicago, IL',
 'London, UK',
 'Portland, OR',
 'Seattle, WA',
 'Austin, TX',
 'Boston, MA',
 'Philadelphia, PA',
 'Minneapolis, MN',
 'Nashville, TN',
 '?',
 'Atlanta, GA',
 'Washington, DC',
 'San Diego, CA',
 'Denver, CO',
 'Toronto, Canada',
 'Manhattan, NY']

In [155]:
dbc5.loc[~dbc5.location.isin(topl), 'location'] = 'other'

In [156]:
dbc5.location.unique()

array(['other', 'Washington, DC', 'Chicago, IL', 'Seattle, WA',
       'Minneapolis, MN', 'New York, NY', 'Boston, MA', 'Brooklyn, NY',
       'Portland, OR', 'San Francisco, CA', 'Austin, TX',
       'Los Angeles, CA', 'Manhattan, NY', 'Atlanta, GA', 'Denver, CO',
       'San Diego, CA', 'Philadelphia, PA', 'Nashville, TN',
       'Toronto, Canada', '?', 'London, UK'], dtype=object)

In [157]:
db_hot = pd.get_dummies(dbc5, drop_first=True, columns=['category', 'currency', 'goal', 'staff_pick','subcategory', 'location'])

In [158]:
X_train, X_test, y_train, y_test = train_test_split(db_hot.drop(['project_id','funded'], axis=1), db_hot['funded'])

In [159]:
LogReg = LogisticRegression(solver='lbfgs')
LogReg.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [160]:
print(LogReg.score(X_train,y_train))

0.7982133333333333


In [161]:
# increasing n top locations to 40

In [162]:
dbc6 = db.copy()

In [163]:
dbc6 = dbc6[['project_id', 'category', 'currency', 'goal', 'staff_pick', 'funded',
       'subcategory', 'location']]

In [164]:
dbc6.columns

Index(['project_id', 'category', 'currency', 'goal', 'staff_pick', 'funded',
       'subcategory', 'location'],
      dtype='object')

In [352]:
#now we wil attempt to create new feattre drived from location
# get top 40 most frequent funded location
n = 40
topl=dbFunded['location'].value_counts()[:n].index.tolist()
topl

['New York, NY',
 'Los Angeles, CA',
 'Brooklyn, NY',
 'San Francisco, CA',
 'Chicago, IL',
 'London, UK',
 'Portland, OR',
 'Seattle, WA',
 'Austin, TX',
 'Boston, MA',
 'Philadelphia, PA',
 'Minneapolis, MN',
 'Nashville, TN',
 '?',
 'Atlanta, GA',
 'Washington, DC',
 'San Diego, CA',
 'Denver, CO',
 'Toronto, Canada',
 'Manhattan, NY',
 'Dallas, TX',
 'Oakland, CA',
 'Pittsburgh, PA',
 'Salt Lake City, UT',
 'Houston, TX',
 'Detroit, MI',
 'Baltimore, MD',
 'New Orleans, LA',
 'Vancouver, Canada',
 'Phoenix, AZ',
 'Orlando, FL',
 'Las Vegas, NV',
 'Montreal, Canada',
 'St. Louis, MO',
 'Columbus, OH',
 'Kansas City, MO',
 'Edinburgh, UK',
 'Paris, France',
 'Miami, FL',
 'Sydney, AU']

In [353]:
dbc6.loc[~dbc6.location.isin(topl), 'location'] = 'other'

In [354]:
dbc6.location.unique()

array(['other', 'Washington, DC', 'Houston, TX', 'Chicago, IL',
       'Seattle, WA', 'Minneapolis, MN', 'New York, NY', 'Las Vegas, NV',
       'Boston, MA', 'Brooklyn, NY', 'Vancouver, Canada', 'Portland, OR',
       'San Francisco, CA', 'Austin, TX', 'St. Louis, MO',
       'Paris, France', 'Los Angeles, CA', 'Manhattan, NY',
       'New Orleans, LA', 'Phoenix, AZ', 'Atlanta, GA', 'Pittsburgh, PA',
       'Denver, CO', 'Oakland, CA', 'Detroit, MI', 'Dallas, TX',
       'Kansas City, MO', 'San Diego, CA', 'Philadelphia, PA',
       'Nashville, TN', 'Baltimore, MD', 'Miami, FL', 'Orlando, FL',
       'Toronto, Canada', 'Edinburgh, UK', 'Columbus, OH', '?',
       'London, UK', 'Salt Lake City, UT', 'Sydney, AU',
       'Montreal, Canada'], dtype=object)

In [355]:
db_hot = pd.get_dummies(dbc6, drop_first=True, columns=['category', 'currency', 'goal', 'staff_pick','subcategory', 'location'])

In [356]:
X_train, X_test, y_train, y_test = train_test_split(db_hot.drop(['project_id','funded'], axis=1), db_hot['funded'])

In [357]:
LogReg = LogisticRegression(solver='lbfgs')
LogReg.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [358]:
print(LogReg.score(X_train,y_train))

0.7988933333333333


In [359]:
db.location.nunique()

10273

In [360]:
#trying to increase topl n to 100

In [406]:
dbc7 = db.copy()

In [407]:
dbc7 = dbc7[['project_id', 'category', 'currency', 'goal', 'staff_pick', 'funded',
       'subcategory', 'location']]

In [408]:
dbc7.columns

Index(['project_id', 'category', 'currency', 'goal', 'staff_pick', 'funded',
       'subcategory', 'location'],
      dtype='object')

In [409]:
dbFunded = db.loc[db['funded'] == True]

In [410]:
#now we wil attempt to create new feattre drived from location
# get top 100 most frequent funded location
n = 100
topl=dbFunded['location'].value_counts()[:n].index.tolist()
topl

['New York, NY',
 'Los Angeles, CA',
 'Brooklyn, NY',
 'San Francisco, CA',
 'Chicago, IL',
 'London, UK',
 'Portland, OR',
 'Seattle, WA',
 'Austin, TX',
 'Boston, MA',
 'Philadelphia, PA',
 'Minneapolis, MN',
 'Nashville, TN',
 '?',
 'Atlanta, GA',
 'Washington, DC',
 'San Diego, CA',
 'Denver, CO',
 'Toronto, Canada',
 'Manhattan, NY',
 'Dallas, TX',
 'Oakland, CA',
 'Pittsburgh, PA',
 'Salt Lake City, UT',
 'Houston, TX',
 'Detroit, MI',
 'Baltimore, MD',
 'New Orleans, LA',
 'Vancouver, Canada',
 'Phoenix, AZ',
 'Orlando, FL',
 'Las Vegas, NV',
 'Montreal, Canada',
 'St. Louis, MO',
 'Columbus, OH',
 'Kansas City, MO',
 'Edinburgh, UK',
 'Paris, France',
 'Miami, FL',
 'Sydney, AU',
 'Melbourne, AU',
 'Tucson, AZ',
 'Sacramento, CA',
 'Cleveland, OH',
 'Richmond, VA',
 'Berlin, Germany',
 'Grand Rapids, MI',
 'San Jose, CA',
 'Cincinnati, OH',
 'Providence, RI',
 'Hollywood, Los Angeles, CA',
 'Boulder, CO',
 'Berkeley, CA',
 'Indianapolis, IN',
 'Madison, WI',
 'Tampa, FL',
 'Mil

In [411]:
dbc7.loc[~dbc7.location.isin(topl), 'location'] = 'other'

In [412]:
dbc7.location.unique()

array(['other', 'Washington, DC', 'Houston, TX', 'Chicago, IL',
       'Seattle, WA', 'Minneapolis, MN', 'New York, NY', 'Raleigh, NC',
       'Las Vegas, NV', 'Boston, MA', 'Brooklyn, NY', 'Savannah, GA',
       'Providence, RI', 'Vancouver, Canada', 'Portland, OR',
       'Durham, NC', 'San Francisco, CA', 'Santa Cruz, CA', 'Austin, TX',
       'St. Louis, MO', 'Richmond, VA', 'Madison, WI', 'Paris, France',
       'Los Angeles, CA', 'Charleston, SC', 'Manhattan, NY',
       'New Orleans, LA', 'Cleveland, OH', 'Cincinnati, OH',
       'Phoenix, AZ', 'Atlanta, GA', 'Rochester, NY', 'Pittsburgh, PA',
       'Portland, ME', 'St. Paul, MN', 'Buffalo, NY', 'Louisville, KY',
       'Lower East Side, Manhattan, NY', 'Santa Monica, CA', 'Denver, CO',
       'Mexico City, Mexico', 'Burlington, VT', 'Oakland, CA',
       'Detroit, MI', 'Albuquerque, NM', 'Dallas, TX', 'Kansas City, MO',
       'Tucson, AZ', 'Sacramento, CA', 'San Diego, CA',
       'Philadelphia, PA', 'Santa Barbara, CA', 'Cam

In [413]:
dbc7.goal

0         2800.0
1         4500.0
2         5000.0
3         6500.0
4        15000.0
          ...   
99995     7000.0
99996    25000.0
99997     5500.0
99998    25000.0
99999     2500.0
Name: goal, Length: 100000, dtype: float64

In [414]:
# encoding goal values into ascending intervals
dbc7['goal'] = pd.cut(dbc7.goal,bins=[0,5000,10000,25000,50000,75000,100000,250000,500000,750000,1000000,2500000,5000000,7500000,10000000,25000000,50000000,75000000,100000000000],
                      labels=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18'])

In [415]:
dbc7.goal

0        1
1        1
2        1
3        2
4        3
        ..
99995    2
99996    3
99997    2
99998    3
99999    1
Name: goal, Length: 100000, dtype: category
Categories (18, object): ['1' < '2' < '3' < '4' ... '15' < '16' < '17' < '18']

In [416]:
db_hot = pd.get_dummies(dbc7, drop_first=True, columns=['category', 'currency', 'goal', 'staff_pick','subcategory', 'location'])

In [417]:
X_train, X_test, y_train, y_test = train_test_split(db_hot.drop(['project_id','funded'], axis=1), db_hot['funded'])

In [418]:
LogReg = LogisticRegression(solver='lbfgs',max_iter=1000, verbose=True)
LogReg.fit(X_train, y_train)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.3s finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=1000,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=True,
                   warm_start=False)

In [419]:
print(LogReg.score(X_train,y_train))

0.79084


80% accuracy score is NICE

In [420]:
# Import the test_set

db_Test = pd.read_csv("/content/drive/MyDrive/IDS/KS_test_data.csv",delimiter=';')

In [421]:
db_Test1 = db_Test.copy()

In [422]:
db_Test1 = db_Test1[['project_id', 'category', 'currency', 'goal', 'staff_pick','subcategory', 'location']]

In [423]:
db_Test1.head()

,project_id,category,currency,goal,staff_pick,subcategory,location
0,KS_100000,art,USD,3900.0,False,art,"Bakersfield, CA"
1,KS_100001,art,USD,750.0,False,painting,"Lowell, MA"
2,KS_100002,art,USD,1000.0,False,painting,"Portland, OR"
3,KS_100003,art,USD,15000.0,False,public art,"Imlay, NV"
4,KS_100004,art,USD,4200.0,True,digital art,"Fredericksburg, VA"


In [424]:
db_Test1.describe()

,goal
count,7.806500e+04
mean,3.925172e+04
std,9.383731e+05
min,1.000000e-02
25%,1.500000e+03
50%,5.000000e+03
75%,1.200000e+04
max,1.000000e+08


In [425]:
db_Test1['goal'] = pd.cut(db_Test1.goal,bins=[0,5000,10000,25000,50000,75000,100000,250000,500000,750000,1000000,2500000,5000000,7500000,10000000,25000000,50000000,75000000,100000000000],
                      labels=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18'])

In [426]:
db_Test1['goal'].isnull().sum()

0

In [427]:
db_Test1.loc[~db_Test1.location.isin(topl), 'location'] = 'other'

In [428]:
db_Test1.location.unique()

array(['other', 'Portland, OR', 'Minneapolis, MN',
       'Hollywood, Los Angeles, CA', 'Houston, TX', 'Los Angeles, CA',
       'Washington, DC', 'Ann Arbor, MI', 'Sacramento, CA', 'Buffalo, NY',
       'Baltimore, MD', 'Seattle, WA', 'Boise, ID', 'Richmond, VA',
       'London, UK', 'Denver, CO', 'Pittsburgh, PA', 'Chicago, IL',
       'Brooklyn, NY', 'New York, NY', 'Atlanta, GA', 'Miami, FL',
       'Rochester, NY', 'Tucson, AZ', 'Burlington, VT', 'Phoenix, AZ',
       'Austin, TX', 'New Orleans, LA', 'San Francisco, CA',
       'Santa Fe, NM', 'Charlotte, NC', 'Milwaukee, WI', 'Savannah, GA',
       'Madison, WI', 'Boston, MA', 'Detroit, MI', 'Asheville, NC',
       'Berkeley, CA', 'Lower East Side, Manhattan, NY', 'Long Beach, CA',
       'Paris, France', 'Philadelphia, PA', 'Boulder, CO',
       'Cleveland, OH', 'Nashville, TN', 'Kansas City, MO', 'Durham, NC',
       'Berlin, Germany', 'Stockholm, Sweden', 'Dallas, TX',
       'Providence, RI', 'Albuquerque, NM', 'Manhattan, NY

In [429]:
db_hot_Test1 = pd.get_dummies(db_Test1, drop_first=True, columns=['category', 'currency', 'goal', 'staff_pick','subcategory', 'location'])

In [430]:
db_Test1.columns

Index(['project_id', 'category', 'currency', 'goal', 'staff_pick',
       'subcategory', 'location'],
      dtype='object')

In [431]:
dbc7.columns

Index(['project_id', 'category', 'currency', 'goal', 'staff_pick', 'funded',
       'subcategory', 'location'],
      dtype='object')

In [432]:
db_hot_Test1.columns

Index(['project_id', 'category_comics', 'category_crafts', 'category_dance',
       'category_design', 'category_fashion', 'category_film & video',
       'category_food', 'category_games', 'category_journalism',
       ...
       'location_St. Paul, MN', 'location_Stockholm, Sweden',
       'location_Sydney, AU', 'location_Tampa, FL', 'location_Tokyo, Japan',
       'location_Toronto, Canada', 'location_Tucson, AZ',
       'location_Vancouver, Canada', 'location_Washington, DC',
       'location_other'],
      dtype='object', length=303)

In [433]:
column_name=['project_id','prediction']
db_hot_pred= pd.DataFrame(columns=column_name) 

In [434]:
db_hot_pred['project_id']=db_hot_Test1.project_id

In [435]:
db_hot_pred

,project_id,prediction
0,KS_100000,NaN
1,KS_100001,NaN
2,KS_100002,NaN
3,KS_100003,NaN
4,KS_100004,NaN
...,...,...
78060,KS_178060,NaN
78061,KS_178061,NaN
78062,KS_178062,NaN
78063,KS_178063,NaN


In [436]:
LogReg.predict(db_hot_Test1.drop(['project_id'], axis=1))

array([ True,  True,  True, ...,  True,  True, False])

In [437]:
db_hot_pred['prediction'] = LogReg.predict(db_hot_Test1.drop(['project_id'], axis=1))

In [438]:
db_hot_pred['prediction']

0         True
1         True
2         True
3        False
4         True
         ...  
78060    False
78061    False
78062     True
78063     True
78064    False
Name: prediction, Length: 78065, dtype: bool

In [439]:
db_hot_pred.to_csv('predictions.tsv', sep='\t', columns=['project_id','prediction'])

In [440]:
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from numpy import mean
from numpy import std

In [441]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

# evaluate model
scores = cross_val_score(LogReg, X_test, y_test, scoring='accuracy', cv=cv, n_jobs=-1)


In [442]:
# report performance
print('Accuracy: %.3f (%.3f)' % (mean(scores), std(scores)))

Accuracy: 0.780 (0.010)


In [443]:
# accuracy times number of data points in test set
0.781 * 78065


60968.765